TryPowerShellメモ

### エンコードのメモ

In [1]:
# 初期設定を保存・確認
Write-Output "`$OutputEncoding: $($OutputEncoding.EncodingName)"
$defaultOutputEncoding = $OutputEncoding
Write-Output "System.Console.OutputEncoding: $([System.Console]::OutputEncoding.EncodingName)"
$defaultConsoleOutputEncoding = [System.Console]::OutputEncoding

$OutputEncoding: Unicode (UTF-8)
System.Console.OutputEncoding: Unicode (UTF-8)


In [2]:
# SJISテストデータ作成

$OutputEncoding = [System.Text.Encoding]::GetEncoding('shift_jis')
& "$($env:OneDrive)\Dev\TryWindowsBatch\Bat\DummyCsv.bat" 10 > ".\work\dummy_1KB.csv"
$OutputEncoding = $defaultOutputEncoding
#=> 文字化けしてしまった。

[System.Console]::OutputEncoding = [System.Text.Encoding]::GetEncoding("shift_jis")
& "$($env:OneDrive)\Dev\TryWindowsBatch\Bat\DummyCsv.bat" 1000 | Out-File -Encoding shift_jis ".\work\dummy_100KB.csv"
[System.Console]::OutputEncoding = $defaultConsoleOutputEncoding

2023/09/09 21:18:03.84 DummyCsv �����J�n���܂��B 
2023/09/09 21:18:03.85 DummyCsv ����I���ł��B 
2023/09/09 21:18:04.04 DummyCsv 処理開始します。 
2023/09/09 21:18:04.40 DummyCsv 正常終了です。 


In [30]:
# SJISファイルを読み込んでみる

$OutputEncoding = [System.Text.Encoding]::GetEncoding('shift_jis')
Select-String '1234,' ".\work\dummy_100KB.csv"
$OutputEncoding = $defaultOutputEncoding
#=> 文字化け

[System.Console]::OutputEncoding = [System.Text.Encoding]::GetEncoding("shift_jis")
Select-String '1234,' ".\work\dummy_100KB.csv"
[System.Console]::OutputEncoding = $defaultConsoleOutputEncoding
#=> 文字化け

Get-Content ".\work\dummy_100KB.csv" -Encoding sjis | Select-String '1234,'


work\dummy_100KB.csv:602:602,0123456789,"abcde","ABCDE","���{��e�L�X�g",12,1234,xxxxxxxxxxxxxxxxxxx
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
work\dummy_100KB.csv:602:602,0123456789,"abcde","ABCDE","���{��e�L�X�g",12,1234,xxxxxxxxxxxxxxxxxxx
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
602,0123456789,"abcde","ABCDE","日本語テキスト",12,1234,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
xxxxxxx



### テキスト処理

In [12]:
# 複数行のマッチングを行う正規表現
$text = "1行目`r`n2行目`r`n3行目`r`n"
Write-Output $text
if ($text -match "2行目[\s\S]+行") {
  Write-Output $Matches[0]
}

1行目
2行目
3行目

2行目
3行


### クリップボード監視

In [ ]:
Add-Type -AssemblyName System.Windows.Forms
#=> Polyglot Notebooks だとエラーになった。
# Add-Type: Cannot find path 'xxx\System.Windows.Forms.dll' because it does not exist.

# While ($true) {
#   $fileList = [System.Windows.Forms.Clipboard]::GetFileDropList()
#   $image = [System.Windows.Forms.Clipboard]::GetImage()
#   $text = [System.Windows.Forms.Clipboard]::GetText()
#   if ($fileList -ne $oldFileList) {
#     Write-Output $fileList
#   }
#   if ($image -ne $oldImage) {
#     Write-Output $image
#   }
#   if ($text -ne $oldText) {
#     Write-Output $text
#   }
#   $oldFileList= $fileList
#   $oldImage = $image
#   $oldText = $text
#   Start-Sleep 1
# }

### 音声合成

In [6]:
# PowerShellでMacのsayコマンドのように合成音声にテキストを読み上げさせてみる | 俺的備忘録 〜なんかいろいろ〜
# https://orebibou.com/ja/home/201708/20170813_001/

Add-Type -AssemblyName System.speech
$ss = New-Object System.Speech.Synthesis.SpeechSynthesizer
$ss.Speak("こんにちは")

$ss.GetInstalledVoices() | %{ $_.VoiceInfo }


Gender                : Female
Age                   : Adult
Name                  : Microsoft Haruka Desktop
Culture               : ja-JP
Id                    : TTS_MS_JA-JP_HARUKA_11.0
Description           : Microsoft Haruka Desktop - Japanese
SupportedAudioFormats : {}
AdditionalInfo        : {[Age, Adult], [Gender, Female], [Language, 411], [Name, Microsoft Haruka D
                        esktop]…}

Gender                : Female
Age                   : Adult
Name                  : Microsoft Zira Desktop
Culture               : en-US
Id                    : TTS_MS_EN-US_ZIRA_11.0
Description           : Microsoft Zira Desktop - English (United States)
SupportedAudioFormats : {}
AdditionalInfo        : {[Age, Adult], [Gender, Female], [Language, 409], [Name, Microsoft Zira Des
                        ktop]…}




### Visual Studio Code

tasks.json
- コマンドレットから"Tasks: Configure Task">"テンプレートから tasks.json を生成">"Others"を選んで生成できた。
- または、メニューの"ターミナル">"タスクの構成"からも生成できる。

ps1ファイル
- F8 - 選択範囲を実行
- Ctrl+F1 - カーソルのコマンドレットについてオンラインヘルプを参照